In [14]:
import pandas as pd
import openai
import time
import json

# 设置您的OpenAI API密钥。
openai.api_key = 'YOUR API KEY'

# 读取数据集
df = pd.read_parquet('Cuted_TD/subset_6.parquet')

# 开始计时
start_time = time.time()

# 打开一个文件以准备写入
with open('qa_pairs6.json', 'w', encoding='utf-8') as json_file:
    json_file.write("[\n")  # 开始一个JSON数组

    first_item = True  # 用于跟踪是否是第一个条目，以避免在第一个条目前添加逗号

    for index, row in df.iterrows():
        data_id = row['ID']
        title = row['title']
        abstract = row['abstract']
        content = title + "." + abstract  # 合并title和abstract为content

        
                
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Given the title: \"{title}\" and abstract: \"{abstract}\". Given the above content, ask an AI-related question based on the title and generate an answer based on the abstract in JSON format [question: What/How xxx ?, answer: xxx. ].you need to put only quotation marks around the contents of the question and answer.The question should be less than 25 tokens and the answer between 100 to 200 tokens."}
        ]

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )


        response_content = response.choices[0].message['content']

        try:
            # 这里添加代码解析response_content中的问题和答案
            qa_content = eval(response_content)
        except:
            qa_content = {"question": "", "answer": ""}  # 如果解析失败，使用空字符串

        qa_pair_with_id = {
            "id": data_id,
            "context": content,
            "question": qa_content["question"],
            "answer": qa_content["answer"]
        }

        # 如果不是第一项，添加一个逗号来分隔项目
        if not first_item:
            json_file.write(",\n")
        else:
            first_item = False

        # 将新的问题答案对（包含ID和content）写入文件
        json.dump(qa_pair_with_id, json_file, ensure_ascii=False, indent=4)

    json_file.write("\n]")  # 结束JSON数组

# 打印用时
print("Total time:", time.time() - start_time)


RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-6QBVtVwtyUGQzVXoQ7ScNa1d on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.

In [26]:
import pandas as pd
import openai
import time
import json

# 设置您的OpenAI API密钥。
openai.api_key = 'YOUR API KEY'
# openai.api_key = 'YOUR API KEY'

# 读取数据集
df = pd.read_parquet('Cuted_TD/subset_6.parquet')

# 开始计时
start_time = time.time()

# 找到ID为21727的数据的索引
start_index = df.index[df['ID'] == 74028].tolist()#断开的ID加1
if start_index:
    start_index = start_index[0]  # 获取ID为21727的数据的索引
else:
    print("ID not found.")
    start_index = len(df)  # 如果没有找到，不处理任何数据

# 从ID为21727的数据开始处理
df = df.iloc[start_index:]

# 以读取模式打开现有的JSON文件来获取内容
try:
    with open('qa_pairs6.json', 'r', encoding='utf-8') as file:
        content = file.read().rstrip('\n')
        # 检查是否需要在末尾添加逗号
        needs_comma = not content.endswith('[') and content != ""
except FileNotFoundError:
    needs_comma = False  # 文件不存在时，不需要添加逗号

# 再次以追加模式打开文件以添加新内容
with open('qa_pairs6.json', 'a', encoding='utf-8') as json_file:
    if needs_comma:
        # 如果文件已存在内容，先添加逗号和换行
        json_file.write(",\n")

    for index, row in df.iterrows():
        data_id = row['ID']
        title = row['title']
        abstract = row['abstract']
        content = title + "." + abstract
                
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Given the title: \"{title}\" and abstract: \"{abstract}\". Given the above content, ask an AI-related question based on the title and generate an answer based on the abstract in JSON format [question: What/How xxx ?, answer: xxx. ].You need to put only quotation marks around the contents of the question and answer.The question should be less than 25 tokens and the answer between 100 to 200 tokens."}
        ]

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )


        response_content = response.choices[0].message['content']

        try:
            # 这里添加代码解析response_content中的问题和答案
            qa_content = eval(response_content)
        except:
            qa_content = {"question": "", "answer": ""}  # 如果解析失败，使用空字符串

        qa_pair_with_id = {
            "id": data_id,
            "context": content,
            "question": qa_content["question"],
            "answer": qa_content["answer"]
        }

        # 写入新的数据对象
        json.dump(qa_pair_with_id, json_file, ensure_ascii=False, indent=4)
        json_file.write(",\n")  # 每个对象后添加逗号和换行符

    # 由于是追加模式，我们在文件最后处理结束符
    # 不需要回退和删除最后的逗号，因为我们将在下一次追加之前处理它

# 在所有数据处理完毕后，关闭数组（如果需要的话可以在下一次打开文件时处理）
# 打印用时
print("Total time:", time.time() - start_time)


Total time: 121.6242618560791
